## ベクトルデータをDBに登録する

In [1]:
import os
import shutil
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings import InfinityEmbeddings

## 定数定義

In [2]:
EMBEDDING_MODEL_PATH = 'sbintuitions/sarashina-embedding-v2-1b'

PDF_PATH = '/app/data/input.pdf'
DB_DIR = '/app/chroma_db'

## DB作成

In [3]:
# PDF読み込み
loader = PyPDFLoader(PDF_PATH)
documents = loader.load()

# チャンク分割
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # 1つのチャンクの文字数
    chunk_overlap=200     # チャンク間の重複文字数
)
splits = text_splitter.split_documents(documents)
enhanced_splits = []
for doc in splits:
    # 元の文書内容にクエリテンプレートを追加
    query_enhanced_content = f"text: {doc.page_content}"
    
    # 新しいDocumentオブジェクトを作成
    from langchain_core.documents import Document
    enhanced_doc = Document(
        page_content=query_enhanced_content,
        metadata=doc.metadata
    )
    enhanced_splits.append(enhanced_doc)
splits = enhanced_splits

# 埋め込み
embeddings = InfinityEmbeddings(
    model=EMBEDDING_MODEL_PATH,
    infinity_api_url='http://proxy',
)
if os.path.exists(DB_DIR):
    shutil.rmtree(DB_DIR)
    print("既存のデータベースを削除しました")

# ベクトルDB作成
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory=DB_DIR
)

既存のデータベースを削除しました


In [4]:
test_results = vectorstore.similarity_search("task: クエリに関連した文章を検索してください \\n query: 生成AIとは?", k=3)
for result in test_results:
    print("-" * 100)
    print(result.page_content)

----------------------------------------------------------------------------------------------------
text: 7 
用語 解説 
生成 AI 出力が単一の数値やラベルではなく、文章や画像や音声や動画といっ
た従来は人間が創作していた類のものを出力する機械学習モデルを使っ
たシステムのことを総称したもの。 
テキスト生成
AI 
生成 AI のうち、出力が文章に代表されるテキストのもの。出力は文章
以外でもソースコード等やテキストデータの形式を変換したものといっ
た自然言語以外の出力や、文章分類や固有表現抽出等といった従来の自
然言語処理課題を行うことも可能である。 
大規模言語モ
デル 
昨今のテキスト生成 AI の背後にいることの多いニューラルネットワー
ク型の機械学習モデルの一種で、膨大なパラメータ数と膨大なテキスト
により学習されていることが特徴の一つ。 
テキスト生成 AI の背後は必ずしも大規模言語モデルではない（機械翻
訳での LSTM 等）点や、大規模言語モデルの小型化研究が盛んであり必ず
しも「大規模」とは限らない点が混乱しやすい。 
プロンプト 大規模言語モデルへ入力するテキストデータのこと。 
膨大なテキストによる教師なし学習を事前学習とよび、特定の課題（下
流タスクとも呼ぶ）に特化させた事後学習(ファインチューニング)とよ
ぶ。以前はこの事後学習によりさまざまな自然言語処理課題に対応させ
る手法が主流だったが、大規模言語モデルではプロンプトによる指示を
変えるだけでさまざまな自然言語処理課題に対応することが可能になっ
た。 
トークン 自然言語を機械学習モデル等のシステムに扱いやすい単位で分割した
もの。日本語の場合、検索エンジンへは n-gram や形態素といった単位で
トークンが作られることが多く、大規模言語モデルへはサブワードとい
ったさらに細かい分割が一般に行われる。 
表２ 本文書の用語集
----------------------------------------------------------------------------------------------------
text: 10 
ここでは、情報検索サービ